In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import os

In [2]:
# Copied all files to:
allL2Dir = '/disk02/UNCERTAINTIES/Monte-Carlo/Matchups/L2/AllFiles/'
# Goal, load l2 files sequentially
# in each case, find out whether latitude/longitude checks out -> re-read Proctor's email
# in any case get median rrs_vvv and rrs_unc_vvv, 
# then compare to the tabulated matchup data incl. in_situ_rrs_vvv
# and seawifs_rrs_vvv

In [3]:
df = pd.read_pickle('dfMatchUpSWF.pkl')

In [4]:
df.head()

,cruise_name,file_name,date_time,latitude,longitude,seawifs_npixel,seawifs_tdiff,seawifs_solz,seawifs_senz,seawifs_cv,...,insitu_Rrs510,insitu_Rrs555,insitu_Rrs670,seawifs_Rrs412,seawifs_Rrs443,seawifs_Rrs490,seawifs_Rrs510,seawifs_Rrs555,seawifs_Rrs670,L1_name
id,,,,,,,,,,,,,,,,,,,,,
31,s990610w,S1999161164837.L2_MLAC.R0000021509_43N_43N_68W...,1999/06/10 14:30:00,43.695,-68.040,100,9180,21.700,22.172,0.0170,...,0.005483,-999.000000,-999.000000,0.002981,0.003594,0.004094,0.003724,0.002715,0.000317,S1999161164837.L1A_MLAC.R0000021509_43N_43N_68...
32,s990610w,S1999161164837.L2_MLAC.R0000021509_43N_43N_68W...,1999/06/10 16:00:00,43.679,-68.646,100,3780,21.533,23.056,0.0510,...,0.004477,0.004331,0.001113,0.001943,0.002601,0.003336,0.003321,0.003147,0.000510,S1999161164837.L1A_MLAC.R0000021509_43N_43N_68...
99,s990824w,S1999236162323.L2_MLAC.R0000021516_43N_43N_68W...,1999/08/24 17:30:00,43.713,-67.756,100,-3300,32.622,46.042,0.0517,...,0.005878,0.004736,-999.000000,0.004441,0.004172,0.003767,0.002986,0.002044,0.000129,S1999236162323.L1A_MLAC.R0000021516_43N_43N_68...
125,s990802w,S1999214163307.L2_MLAC.R0000021513_43N_43N_68W...,1999/08/02 16:00:00,43.745,-66.986,100,2760,26.128,31.354,0.0388,...,0.004385,-999.000000,-999.000000,0.002260,0.002420,0.002716,0.002618,0.002335,0.000418,S1999214163307.L1A_MLAC.R0000021513_43N_43N_68...
228,s000712w,S2000194161838.L2_MLAC.R0000021391_43N_43N_68W...,2000/07/12 14:30:00,43.755,-66.603,100,7380,21.903,42.950,0.0395,...,0.003403,0.002944,0.000443,0.002083,0.002601,0.003412,0.003169,0.002577,0.000319,S2000194161838.L1A_MLAC.R0000021391_43N_43N_68...


In [6]:
df.columns

Index(['cruise_name', 'file_name', 'date_time', 'latitude', 'longitude',
       'seawifs_npixel', 'seawifs_tdiff', 'seawifs_solz', 'seawifs_senz',
       'seawifs_cv', 'seawifs_es_thresh', 'seawifs_windspeed', 'insitu_Rrs412',
       'insitu_Rrs443', 'insitu_Rrs490', 'insitu_Rrs510', 'insitu_Rrs555',
       'insitu_Rrs670', 'seawifs_Rrs412', 'seawifs_Rrs443', 'seawifs_Rrs490',
       'seawifs_Rrs510', 'seawifs_Rrs555', 'seawifs_Rrs670', 'L1_name'],
      dtype='object')

In [20]:
df.filter(items=['file_name','L1_name'],axis=1).head(10)

,file_name,L1_name
id,,
31,S1999161164837.L2_MLAC.R0000021509_43N_43N_68W...,S1999161164837.L1A_MLAC.R0000021509_43N_43N_68...
32,S1999161164837.L2_MLAC.R0000021509_43N_43N_68W...,S1999161164837.L1A_MLAC.R0000021509_43N_43N_68...
99,S1999236162323.L2_MLAC.R0000021516_43N_43N_68W...,S1999236162323.L1A_MLAC.R0000021516_43N_43N_68...
125,S1999214163307.L2_MLAC.R0000021513_43N_43N_68W...,S1999214163307.L1A_MLAC.R0000021513_43N_43N_68...
228,S2000194161838.L2_MLAC.R0000021391_43N_43N_68W...,S2000194161838.L1A_MLAC.R0000021391_43N_43N_68...
229,S2000194161838.L2_MLAC.R0000021391_43N_43N_68W...,S2000194161838.L1A_MLAC.R0000021391_43N_43N_68...
245,S2000193171348.L2_MLAC.R0000021390_43N_43N_69W...,S2000193171348.L1A_MLAC.R0000021390_43N_43N_69...
258,S2000184171542.L2_MLAC.R0000021389_43N_43N_69W...,S2000184171542.L1A_MLAC.R0000021389_43N_43N_69...
259,S2000184171542.L2_MLAC.R0000021389_43N_43N_69W...,S2000184171542.L1A_MLAC.R0000021389_43N_43N_69...
